In [49]:
import datetime as dt
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
effis = gpd.read_file('~/Descargas/effis_layer/modis.ba.poly.shp')
effis.head()

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,SCLEROPH,TRANSIT,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,geometry
0,264477,2000-09-20 00:00:00,2025-03-28 15:01:38.620495,EL,??????,???????? ????????? ?????,981,2.1472392638034616,0,0,5.4192229038849264,73.51738241308041,1.431492842535641,16.46216768915987,1.0224948875254578,0,32.02719647413729,FireSeason,"POLYGON ((24.10056 38.66232, 24.10079 38.65254..."
1,260352,2000-07-31 00:00:00,2000-12-31 12:10:11.414,IT,Crotone,Roccabernarda,56,0,0,0,0,0,58.49056603762549,41.50943396218583,0,0,99.99999999999997,FireSeason,"POLYGON ((16.84555 39.15941, 16.84656 39.15743..."
2,260396,2000-08-25 00:00:00,2000-12-31 12:10:11.414,IT,Crotone,Pallagorio,413,61.36919315401923,0,0,0,0,0,38.63080684595633,0,0,98.7552636258531,FireSeason,"POLYGON ((16.89034 39.33069, 16.89195 39.32436..."
3,260398,2000-07-15 00:00:00,2000-12-31 12:10:11.414,IT,Cosenza,Corigliano-Rossano,88,43.678160919490026,0,0,0,0,0,56.32183908039503,0,0,0,FireSeason,"POLYGON ((16.70885 39.57951, 16.71165 39.57865..."
4,259734,2000-09-02 00:00:00,2000-12-31 12:10:11.414,PT,Beiras e Serra da Estrela,Alvendre,54,0,0,0,0,21.81818181814215,45.45454545446281,32.727272727213226,0,0,0,FireSeason,"POLYGON ((-7.2476 40.59114, -7.24515 40.59178,..."


In [3]:
effis.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 92772 entries, 0 to 92771
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          92772 non-null  object  
 1   FIREDATE    92772 non-null  object  
 2   LASTUPDATE  92772 non-null  object  
 3   COUNTRY     92764 non-null  object  
 4   PROVINCE    92772 non-null  object  
 5   COMMUNE     92772 non-null  object  
 6   AREA_HA     92772 non-null  object  
 7   BROADLEA    92121 non-null  object  
 8   CONIFER     92121 non-null  object  
 9   MIXED       92121 non-null  object  
 10  SCLEROPH    92121 non-null  object  
 11  TRANSIT     92121 non-null  object  
 12  OTHERNATLC  92121 non-null  object  
 13  AGRIAREAS   92121 non-null  object  
 14  ARTIFSURF   92121 non-null  object  
 15  OTHERLC     92121 non-null  object  
 16  PERCNA2K    92772 non-null  object  
 17  CLASS       92772 non-null  object  
 18  geometry    92772 non-null  geometry
d

In [38]:
effis.AREA_HA = effis.AREA_HA.astype(int)
effis = effis[effis.AREA_HA > 499]
effis.AREA_HA.describe()

count     5996.000000
mean      1813.098899
std       3670.876707
min        500.000000
25%        652.000000
50%        926.000000
75%       1635.000000
max      96610.000000
Name: AREA_HA, dtype: float64

In [4]:
# Pasamos las columnas de inicio de incendio y última actualización a datetime 
effis.FIREDATE = pd.to_datetime(effis.FIREDATE, format='ISO8601')
effis.LASTUPDATE = pd.to_datetime(effis.LASTUPDATE, format='ISO8601')

# Eliminamos los incendios de 2025
effis = effis[effis.FIREDATE.dt.year < 2025]

# Calculamos la duración del incendio
effis['DURATION'] = effis.LASTUPDATE - effis.FIREDATE

In [ ]:
# Calculamos el día juliano pero como si todos los años fueran bisiestos

def dia_juliano_bisiesto(serie):
    dias_meses = [0,31,29,31,30,31,30,31,31,30,31,30]
    lista_dias = []
    for valor in serie:
        dia_juliano = 0
        mes = valor.month
        for index in range(mes):
            dia_juliano += dias_meses[index]
        dia_juliano += valor.day
        lista_dias.append(dia_juliano)
    return lista_dias

effis['DAY_YEAR'] = dia_juliano_bisiesto(effis.FIREDATE)

In [50]:
# Calculamos centroides y extraemos coordenadas
effis['CENTROID'] = effis.geometry.centroid.to_crs('WGS 84')
effis['LAT'], effis['LON'] = effis.CENTROID.y, effis.CENTROID.x

/tmp/ipykernel_2719/2172088228.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  effis['CENTROID'] = effis.geometry.centroid.to_crs('WGS 84')
